In [2]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from kor import create_extraction_chain, Object, Text, Option, Selection
import openai

In [3]:
llm = openai.OpenAI(
    base_url="http://localhost:8080/v1",
    api_key = "sk-no-key-required"
)

In [10]:
completion = llm.chat.completions.create(
    model="llava",
    messages=[
        {"role": "system", "content": "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
        {"role": "user", "content": "what human do for fun?"},
    ],
)

message = completion.choices[0].message

In [11]:
print(message)

ChatCompletionMessage(
    content='There are many things that humans do for fun. Some examples include:\n\n1. Watching movies or TV 
shows\n2. Playing video games\n3. Going to concerts or music festivals\n4. Reading books or listening to 
podcasts\n5. Hiking or camping in nature\n6. Traveling to new places\n7. Spending time with friends and family\n8. 
Engaging in sports or physical activities\n9. Cooking or baking\n10. Painting, drawing, or creating other forms of 
art.\n\nThese are just a few examples, but there are countless other ways that humans enjoy themselves and find 
entertainment.',
    role='assistant',
    function_call=None,
    tool_calls=None
)

In [8]:
import requests
import json
import io
import base64

from rich import print
from PIL import Image


class LLAVA:
    SYSTEM_PROMPT = """A chat between a curious human and an artificial intelligence assistant. 
The assistant gives helpful, detailed, and polite answers to the human's questions."""

    def __init__(self):
        self.url = 'http://127.0.0.1:8080/completion'
        self.system_prompt = self.SYSTEM_PROMPT

        self.parameters = {
            "stream": True,
            "n_predict": 400,
            "temperature": 0.2,
            "stop": ["</s>", "Llama:", "User:"],
            "repeat_last_n": 256,
            "repeat_penalty": 1.18,
            "top_k": 40,
            "top_p": 0.7,
            "tfs_z": 1,
            "typical_p": 1,
            "presence_penalty": 0,
            "frequency_penalty": 0,
            "mirostat": 0,
            "mirostat_tau": 5,
            "mirostat_eta": 0.1,
            "grammar": "",
            "n_probs": 0,
            "cache_prompt": True,
            "slot_id": -1,
        }

    def process_image(self, image):
        if image.startswith('http://') or image.startswith('https://'):
            response = requests.get(image)
            response.raise_for_status()
            img_data = io.BytesIO(response.content)
        else:
            img_data = image

        with Image.open(img_data) as img:
            img_bytes = io.BytesIO()
            img.save(img_bytes, format=img.format)
            encoded_img_bytes = base64.b64encode(img_bytes.getvalue()).decode('utf-8')

        return encoded_img_bytes


    def complete(self, image_path, prompt="", stream=False):
        enriched_prompt = f"USER:[img-10]{prompt}\nASSISTANT:"

        encoded_img_bytes = self.process_image(image_path)        
        data = self.parameters | {
            "prompt": enriched_prompt,
            "image_data": [{"data": encoded_img_bytes, "id": 10}]
        }

        response = requests.post(
            url=self.url,
            headers={'Accept': 'text/event-stream'},
            data=json.dumps(data),
            stream=True
        )

        if stream:
            def response_generator():
                for line in response.iter_lines():
                    if line:
                        line_str = line.decode('utf-8')
                        if line_str.startswith('data: '):
                            json_str = line_str[6:]
                            try:
                                json_data = json.loads(json_str)
                                yield json_data["content"]
                            except json.JSONDecodeError:
                                pass
            return response_generator()
        else:
            content = ""
            for line in response.iter_lines():
                if line:
                    line_str = line.decode('utf-8')
                    if line_str.startswith('data: '):
                        json_str = line_str[6:]
                        try:
                            json_data = json.loads(json_str)
                            content += json_data["content"]
                        except json.JSONDecodeError:
                            pass
            return content

In [13]:
llava =  LLAVA()

image = "https://avatars.githubusercontent.com/u/112222186?v=4"

response = llava.complete(image)

print(response)



The image features a person sitting at their computer, with an amusing twist. A green frog is perched on the 
individual's shoulders or head, adding a playful element to the scene. In addition to this unique feature, there 
are two keyboards visible in the picture - one placed directly underneath the person and another towards the right 
side of the image.

In [12]:
llava =  LLAVA()

image = "https://avatars.githubusercontent.com/u/112222186?v=4"

response = llava.complete(image, stream=True)

for chunks in response:
    print(chunks, end=" ")



The

image

features

a

person

...

of

the

f

rog

.